In [1]:
import random
import numpy as np

In [2]:
def MM1(arrival_rate, service_rate, num_customers):
    inter_arrival_times = [random.expovariate(arrival_rate) for _ in range(num_customers)]
    service_times = [random.expovariate(service_rate) for _ in range(num_customers)]
    arrival_times = []
    start_service_times = []
    finish_service_times = []
    waiting_times = []
    
    current_time = 0
    for i in range(num_customers):
        if i == 0:
            arrival_times.append(inter_arrival_times[i])
        else:
            arrival_times.append(arrival_times[i-1] + inter_arrival_times[i])
        
        start_service_time = max(arrival_times[i], current_time)
        start_service_times.append(start_service_time)
        
        finish_service_time = start_service_time + service_times[i]
        finish_service_times.append(finish_service_time)
        
        waiting_time = start_service_time - arrival_times[i]
        waiting_times.append(waiting_time)
        
        current_time = finish_service_time

    avg_waiting_time = sum(waiting_times) / num_customers
    busy_time = sum(service_times)
    total_time = finish_service_times[-1] - arrival_times[0]
    return avg_waiting_time, waiting_times, busy_time, total_time, inter_arrival_times,arrival_times, start_service_times, finish_service_times, service_times


## Simulation Setup

In [3]:
LAMBDA = 2  # Arrival rate
MU = 4      # Service rate
CUSTOMERS = 1000
RO = LAMBDA / MU  # Traffic intensity
avg_waiting_time, waiting_times, busy_time, total_time, inter_arrival_times,arrival_times, start_service_times, finish_service_times, service_times = MM1(LAMBDA,MU,CUSTOMERS)




## Input Validation

In [4]:
print(f"Sample Mean Inter-Arrival Times: {np.mean(np.array(inter_arrival_times))} Error: {abs(np.mean(np.array(inter_arrival_times)) - 1/LAMBDA)}")
print(f"Sample Mean Service Times: {np.mean(np.array(service_times))} Error: {abs(np.mean(np.array(service_times)) - 1/MU)}")
print(f"Average Waiting Time: {avg_waiting_time}")  

Sample Mean Inter-Arrival Times: 0.5151510016300367 Error: 0.015151001630036687
Sample Mean Service Times: 0.2534187019983331 Error: 0.003418701998333107
Average Waiting Time: 0.2094119751562221


## Indicator 1 Utilization Ro

In [5]:
### Indicator 1 Utilization Ro
utilization = busy_time / total_time
print(f"Utilization (Ro): {utilization}, Theoretical Ro: {RO}, Error: {abs(utilization - RO)}")

Utilization (Ro): 0.4915879767212942, Theoretical Ro: 0.5, Error: 0.008412023278705827


## Indicator 2 Average Number in System L

In [6]:
L = RO / (1 - RO)  # Theoretical average number in system
n_duration  = 0
avg_number_in_system = LAMBDA * (avg_waiting_time + 1 / MU)

In [7]:
def time_weighted_L(arrival_times, finish_service_times):
    events = []
    for t in arrival_times:
        events.append((t, +1))
    for t in finish_service_times:
        events.append((t, -1))
    events.sort(key=lambda x: x[0])

    num_in_system = 0
    prev_t = events[0][0]
    area = 0.0

    for t, delta in events:
        dt = t - prev_t
        area += num_in_system * dt
        num_in_system += delta
        prev_t = t

    total_time = events[-1][0] - events[0][0]
    L_time_weighted = area / total_time
    return L_time_weighted
avg_number_in_system = time_weighted_L(arrival_times, finish_service_times)

print(f"Average Number in System (L): {avg_number_in_system}, Theoretical L: {L}, Error: {abs(avg_number_in_system - L)}")

Average Number in System (L): 0.8978105970586613, Theoretical L: 1.0, Error: 0.10218940294133871


## Indicator 3 Time in System W

In [8]:
W = 1 / (MU - LAMBDA)  # Theoretical average time in system
Time_in_system = np.average(np.array(finish_service_times) - np.array(arrival_times))
print(f"Average Time in System (W): {Time_in_system}, Theoretical W: {W}, Error: {abs(Time_in_system - W)}")

Average Time in System (W): 0.4628306771545555, Theoretical W: 0.5, Error: 0.0371693228454445


## Indicator 4 Time in Queu Wq

In [9]:
Wq = W -1 / MU
Time_in_queue = np.average(np.array(start_service_times) - np.array(arrival_times))
print(f"Average Time in Queue (Wq): {Time_in_queue}, Theoretical Wq: {Wq}, Error: {abs(Time_in_queue - Wq)}")

Average Time in Queue (Wq): 0.20941197515622206, Theoretical Wq: 0.25, Error: 0.04058802484377794


## Stage 3 Internal Consistency (Little's Law)

In [10]:
if avg_number_in_system == LAMBDA * Time_in_system:
    print("Little's Law holds: L = λ * W")
else :
    print(f"Little's Law does not hold. Error: {abs(avg_number_in_system - LAMBDA * Time_in_system)}")

Little's Law does not hold. Error: 0.02785075725044972
